## Preamble

In [1]:
from example_helpers import setup_notebook
from pyetm.models import Scenario

setup_notebook()

Environment setup complete
  Using ETM API at     http://localhost:3000/api/v3
  Token loaded?        True
API connection ready


## Picking my scenario

Use an API session ID to load up one of your scenarios.


In [2]:
# Example beta scenario
scenario = Scenario.load(2690456)

# Example pro scenario
# scenario = Scenario.load(1340415)

## Create a Packer

The scenario packer allows you to collect scenarios and define what information you want to extract from all of them. It helps to collect all info into dataframes or Excel.

- **Main info** with `main_info()` you display all meta information about the scenarios, like area code and end years;

- Input settings
    - **Inputs** `inputs()` displays all inputs for the scenario including their unit, default and user-set values;
    - **Custom curves** using `custom_curves()` you can retrieve and display the curves uploaded to the scenario;
    - **Sortables** `sortables()` shows all user sortables set for the scenario, like heat network orders, etc;
- Outputs:
    - **GQuery results** with `gquery_results()` you can retrieve the queries that were prepared for the scenarios;
    - **Output curves** `output_curves()` downloads and shows the carreir curves requested for the scenarios.


Scenarios can be added to a packer in multiple ways:

- `add(*scenarios)`  will ready the scenario for all 6 packs (or Excel tabs) described above
- `add_inputs(*scenarios)` will only ready the scenarios for the inputs dataframe or Excel tab
- `add_sortables(*scenarios)` will only ready the scenarios for the sortables dataframe or Excel tab
- `add_custom_curves(*scenarios)` will only ready the scenarios for the custom_curves dataframe or Excel tab
- `add_output_curves(*scenarios)` will only ready the scenarios for the output_curves dataframe or Excel tab

If a scenario is added in any of the individual packs, it will be automatically added for main info and prepared gqueries.

-----------------

NOTE: the packer currently only works for a single scenario!

-----------------

In [3]:
from pyetm.models import ScenarioPacker

packer = ScenarioPacker()

scenario = Scenario.load(2690456)
scenario2 = Scenario.load(2690455)
# Ready the scenario for inputs
packer.add_inputs(scenario)
packer.add_inputs(scenario2)

# Main info is automatically readied
print(packer.inputs())
print(packer.main_info())


                                                   II3050v2_NAT_2050  \
input                                                                  
agriculture_bio_kerosene_in_crude_oil_share                      0.0   
agriculture_biodiesel_in_crude_oil_share                         0.0   
agriculture_burner_crude_oil_share                               0.0   
agriculture_burner_hydrogen_share                                0.0   
agriculture_burner_network_gas_share                             0.0   
...                                                              ...   
wtp_of_industry_chemicals_other_flexibility_p2h...              25.0   
wtp_of_industry_chemicals_refineries_flexibilit...              25.0   
wtp_of_industry_other_food_flexibility_p2h_elec...              25.0   
wtp_of_industry_other_paper_flexibility_p2h_ele...              25.0   
wtp_of_transport_car_flexibility_p2p_electricity                 5.0   

                                                   II3050v2_EUR

When you add gqueries to a scenario which is connected to the packer, these queries will be readied for the pack.

In [4]:
scenario.add_queries(
    ["dashboard_total_costs", "dashboard_co2_emissions_versus_start_year"]
)

print(packer.gquery_results())

                                                   II3050v2_EUR_2050
                                                              future
gquery                                    unit                      
dashboard_total_costs                     bln_euro         69.189065
dashboard_co2_emissions_versus_start_year factor           -1.026581


Queries can be added on-the-go. When the packer is exported to Excel, all requested queries will show up.

In [5]:
scenario.add_queries(
    ["dashboard_emissions"]
)

scenario2.add_queries(
    ["dashboard_emissions"]
)

# packer.to_excel('my_excel.xlsx')

Output curves:

In [6]:
packer.add_output_curves(scenario)
packer.add_output_curves(scenario2)
packer.add_custom_curves(scenario)
packer.add_custom_curves(scenario2)
packer.add_sortables(scenario)
packer.add_sortables(scenario2)


packer.to_excel('full_scenario.xlsx')